In [ ]:
import os
import re
import pandas as pd
import numpy as np
import yaml
from pathlib import Path
from tqdm import tqdm

/home/krules/micromamba/envs/meld/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-29 20:11:20.609195: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-29 20:11:20.927826: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-29 20:11:20.927884: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-29 20:11:20.945080: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plug

In [2]:
DATA_DIRS = {
    "train": "./data/train/train_splits",
    "val": "./data/dev/dev_splits_complete",
    "test": "./data/test/output_repeated_splits_test"
}

OUTPUT_BASE = "./output"

os.makedirs(OUTPUT_BASE, exist_ok=True)

## Carregamento e Transformação

In [ ]:
def to_seconds(t):
    h, m, s_ms = t.split(":")
    s, ms = s_ms.split(",")
    return int(h)*3600 + int(m)*60 + int(s) + int(ms)/1000

def to_snake_case(name: str) -> str:
    """Convert a string (like a column name) to snake_case."""
    name = name.strip()
    name = re.sub(r"[^\w\s]", "", name)
    name = re.sub(r"\s+", "_", name)
    return name.lower()

def prepare_df(df: pd.DataFrame):
    df["start_s"] = df["StartTime"].apply(to_seconds)
    df["end_s"] = df["EndTime"].apply(to_seconds)
    df["duration_s"] = df["end_s"] - df["start_s"]

    df[df["duration_s"] <= 0].head()

    df = df.drop(columns=["Sr No.", "StartTime", "EndTime", "Season", "Episode", "Sentiment"])

    df.columns = [to_snake_case(c) for c in df.columns]

    return df

def load_embeddings(df, cfg):
    """Return input X and labels y concatenating chosen modalities."""
    X_list = []
    if cfg["modalities"]["text"]["enabled"]:
        X_text = [
            np.load(Path(cfg["modalities"]["text"]["path"]) / f"{r.dialogue_id}_{r.utterance_id}.npy")
            for _, r in df.iterrows()
        ]
        X_list.append(np.stack(X_text))
    if cfg["modalities"]["audio"]["enabled"]:
        X_audio = [
            np.load(Path(cfg["modalities"]["audio"]["path"]) / f"{r.dialogue_id}_{r.utterance_id}.npy")
            for _, r in df.iterrows()
        ]
        X_list.append(np.stack(X_audio))
    if cfg["modalities"]["video"]["enabled"]:
        X_video = [
            np.load(Path(cfg["modalities"]["video"]["path"]) / f"{r.dialogue_id}_{r.utterance_id}.npy")
            for _, r in df.iterrows()
        ]
        X_list.append(np.stack(X_video))

    # concatenate features along dimension 1
    X = np.concatenate(X_list, axis=1)
    y = df["emotion"].astype("category").cat.codes.to_numpy()
    return X, y

In [ ]:
train_df = prepare_df(pd.read_csv("./data/train_sent_emo.csv"))

## Funções base

In [6]:
def load_config(path):
    with open(path, "r") as f:
        return yaml.safe_load(f)

In [7]:
class BaseExtractor:
    def __init__(self, save_dir):
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(parents=True, exist_ok=True)

    def save(self, dialogue_id, utterance_id, vector):
        path = self.save_dir / f"{dialogue_id}_{utterance_id}.npy"
        np.save(path, vector)
        return str(path)